In [7]:
import mysql.connector
from mysql.connector import pooling

In [8]:
class MySQLDatabase:
    def __init__(self, host, database, user, password, pool_name="mypool", pool_size=5):
        """
        Initialize the MySQL database connection pool.
        """
        self.connection_pool = mysql.connector.pooling.MySQLConnectionPool(
            pool_name=pool_name,
            pool_size=pool_size,
            host=host,
            database=database,
            user=user,
            password=password,
        )

    def call_procedure(self, proc_name, args=None):
        """
        Call a stored procedure and handle its output.
        Args:
            proc_name (str): The name of the stored procedure.
            args (tuple): The arguments for the procedure.
        Returns:
            list or str: A list of results for SELECT operations or a single message for status operations.
        """
        connection = self.connection_pool.get_connection()
        try:
            cursor = connection.cursor()
            if args:
                cursor.callproc(proc_name, args)
            else:
                cursor.callproc(proc_name)

            # Retrieve output parameters or results
            results = []
            for result in cursor.stored_results():
                results.append(result.fetchall())

            # Check if there are OUT parameters
            if args and cursor.rowcount == -1:  # For OUT parameters
                return args

            return results

        except mysql.connector.Error as err:
            print(f"Error: {err}")
        finally:
            cursor.close()
            connection.close()

    def insert_data(self, proc_name, args):
        """
        Insert data using the specified stored procedure.
        Args:
            proc_name (str): Name of the stored procedure for insertion.
            args (tuple): Arguments to be passed to the procedure.
        Returns:
            str: Return message from the procedure.
        """
        result = self.call_procedure(proc_name, args)
        if result:
            print(f"Procedure `{proc_name}` executed successfully.")
            return result
        return "Procedure execution failed."

    def get_table_data(self, proc_name, args=None):
        """
        Retrieve table data using the specified stored procedure.
        Args:
            proc_name (str): Name of the stored procedure for retrieval.
            args (tuple): Arguments to be passed to the procedure.
        Returns:
            list: A list of rows fetched from the procedure.
        """
        results = self.call_procedure(proc_name, args)
        if results:
            print(f"Procedure `{proc_name}` executed successfully.")
            return results[0] if results else []
        return []



In [9]:
# Initialize the MySQLDatabase object
db = MySQLDatabase(
    host="localhost",
    database="root",
    user="root",
    password="moving_bazzar"
)

   

ProgrammingError: 1045 (28000): Access denied for user 'root'@'localhost' (using password: YES)

In [ ]:
 # Example 1: Insert data and get a message
    result_msg = db.insert_data("insert_data", ("John Doe", 30))  # Procedure name and args
    print("Insert Procedure Response:", result_msg)

    # Example 2: Retrieve data from the table
    data = db.get_table_data("get_all_data")  # Procedure name
    print("Table Data:")
    for row in data:
        print(row)